# Webhooks Example 
This pipeline shows how to use Indexical's `webhooks` action to use the results from your data extraction pipeline in your business workflows. 

### Setup
In this section, we'll set up the environment to start using Indexical's API by importing the necessary libraries and saving the API key. Start by inputting your API key below. You can generate an API key from the Indexical console by selecting `Keys` and then hitting `New API Key`. 

In [2]:
import requests
import json


In [3]:

API_KEY="<REPLACE WITH YOUR API_KEY>" 

### Add Pipeline
To create scalable data extraction workflows, Indexical allows you to construct a pipeline of high-level steps that describe how to go from a URL / query to clean, structured data. Each step contains an `action` that is tied to a task-specific agent on our end, as well as a natural language `goal` to instruct the agent. We'll start by importing the `search.json` file as our pipeline and printing it below. 

This pipeline will just scrape the webpage into a human-readable markdown format. Note the use of the special value [`$mainContent`](https://docs.indexical.dev/values). 


In [4]:
pipeline_steps = {
    "name": "scrape",
    "steps": [
        {
            "action": "extract",
            "schema": {
                "content": "$mainContent"
            }
        }
    ]
}

Now, we'll call Indexical's `pipelines` API endpoint to save that pipeline for use in future data extraction workflows Each API request requires the header with `x-api-key`, and you include the JSON pipeline as the API body.  

In [5]:
response = requests.post("https://app.indexical.dev/pipelines",
                          headers={'x-api-key': API_KEY}, 
                          json=pipeline_steps)

if response.status_code == 200: 
    # Convert the response to JSON format
    data = response.json()
else:
    print(response.status_code)

### Run data extraction job & send results to webhook
Once you've saved your pipeline, you can then run that workflow on any set ofwebsites / queries through that pipeline. Indexical will use AI to handle the process of mapping that pipeline to relevant selectors and information on each page, gathering and transforming the website into a clean, standardized schema of your choosing. 

In this case, I will run the `scrape` pipeline on 2 distinct URLs, and when the run is completed, I will send the results to a subscriber URL. 

In [14]:
response = requests.post("https://app.indexical.dev/runs",
                          headers={'x-api-key': API_KEY}, 
                          json={
                              "name" : "scrape", 
                              "urls" : ["https://woocommerce.github.io/woocommerce-rest-api-docs", "https://docs.stripe.com/api/"], 
                              "proxiesEnabled" : True, 
                              "onComplete" : {
                                  "behavior" : "transmit", 
                                  "url" : "<INPUT YOUR SUBSCRIBER URL HERE>"
                              } 
                          })



By default, the `runs` endpoint will run the data extraction pipeline and return the results asynchronously (either available for download on the developer console or transmitted to a subscriber URL via webhooks). As a response, the `runs` endpoint will return both the `pipelineID` and `runID`. 

In [15]:
if response.status_code == 200: 
    # Convert the response to JSON format
    data = response.json()
    run_id = data['id']
    print(data)
else:
    print(response.status_code)

{'pipeline': 1022, 'id': 1556}


### Getting Results
To get the results programmatically, you can either use [webhooks](https://docs.indexical.dev/runs) or use the `outputs` endpoint. Simply call `https://app.indexical.dev/runs/:runId/outputs` with the `runID` returned by the `runs` endpoint.  

In [16]:
output_endpoint = "https://app.indexical.dev/runs/" + str(run_id) + "/outputs"
response = requests.get(output_endpoint,
                          headers={'x-api-key': API_KEY})




By default, the results will be a JSON file with 2 keys `results` and `errors`. Each key will contain an array of results. Each result has a `seed` query/URL, the final `url` from which Indexical extracted the information, as well as a `data` key which has all of the data specified in the pipeline. If Indexical was not able to find information on the page that maps to a specific element, it will either return `NULL` or not output that key.   

In [17]:
if response.status_code == 200: 
    # Convert the response to JSON format
    data = response.json()
    print(json.dumps(data, indent=4))
else:
    print(response.status_code)

{
    "results": [
        {
            "id": 7594566,
            "pipeline": 1022,
            "run": 1556,
            "seed": "https://woocommerce.github.io/woocommerce-rest-api-docs",
            "url": "https://woocommerce.github.io/woocommerce-rest-api-docs/#introduction",
            "data": {
                "content": "INTRODUCTION\n\nWooCommerce (WC) 2.6+ is fully integrated with the WordPress REST\n[http://en.wikipedia.org/wiki/Representational_State_Transfer] API. This allows\nWC data to be created, read, updated, and deleted using requests in JSON format\nand using WordPress REST API Authentication methods and standard HTTP verbs\nwhich are understood by most HTTP clients.\n\nThe current WP REST API integration version is v3 which takes a first-order\nposition in endpoints.\n\nThe following table shows API versions present in each major version of\nWooCommerce:\n\nAPI Version WC Version WP Version Documentation v3 3.5.x or later 4.4 or later -\nv2 3.0.x or later 4.4 or l